In [1]:
#See below for observable trends 
#Import pandas library
import pandas as pd
import numpy as np

In [2]:
#Load file 
school_data = "../Resources/schools_complete.csv"
student_data = "../Resources/students_complete.csv"

In [3]:
#Read files and store into df
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)

In [4]:
#Merge files with common element of school name 
school_data_df = pd.merge(student_df, school_df, how = "left", on="school_name")

In [5]:
#District Summary
#Calculate the total number of schools
unique = school_data_df["school_name"].unique()
total_schools = len(unique)

#Calculate the total number of students
total_students = school_data_df["Student ID"].count()

#Calculate the total budget
total_budget = school_df["budget"].sum()

#Calculate the average math score
average_math = school_data_df["math_score"].mean()

#Calculate the average reading score
average_reading = school_data_df["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
passed_math_df = school_data_df.loc[school_data_df["math_score"] >= 70,:]
passing_math = (passed_math_df["Student ID"].count())/(school_data_df["Student ID"].count())*100

#Calculate the percentage of students with a passing reading score (70 or greater)
passed_reading_df = school_data_df.loc[school_data_df["reading_score"] >= 70,:]
passing_reading = (passed_reading_df["Student ID"].count())/(school_data_df["Student ID"].count())*100

#Merge the passed math and passed reading students based on common Student ID 
passed_merge_df = pd.merge(passed_math_df, passed_reading_df, on="Student ID")

#Calculate the percentage of students who passed math and reading (% Overall Passing)
overall_passing = passed_merge_df["Student ID"].count()/(school_data_df["Student ID"].count())*100

#Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({'Total Schools': [total_schools],
                        'Total Students': [total_students],
                        'Total Budget': [total_budget],
                        'Average Math Score': [average_math],
                        'Average Reading Score': [average_reading],
                        '% Passing Math': [passing_math],
                        '% Passing Reading': [passing_reading],
                        '% Overall Passing': [overall_passing]
                      })

#Apply displayed formatting via mapping 
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [6]:
#School Summary 
#Select wanted columns for a reduced dataframe 
reduced_school_df = school_df.loc[:, ['school_name','type','size','budget']]

#Calculate budget per student 
per_studentbudget = reduced_school_df['budget']/reduced_school_df['size']
reduced_school_df["Per Student Budget"] = per_studentbudget

#Group school data by school name 
grouped_schoolname_df = school_data_df.groupby('school_name')

#Calculate average math and reading scores based on school name 
groupedavg_math = grouped_schoolname_df['math_score'].mean()
groupedavg_reading = grouped_schoolname_df['reading_score'].mean()

#Count the students who passed math and reading per school name 
groupedpass_math = student_df.loc[student_df['math_score']>=70,:].groupby('school_name')['math_score'].count()
groupedpass_reading = student_df.loc[student_df['reading_score']>=70,:].groupby('school_name')['reading_score'].count()

#Place above calculated data into dataframe  
school_summary_df = pd.DataFrame({'Average Math Score': groupedavg_math,
                                   'Average Reading Score': groupedavg_reading,
                                  'Passing Reading': groupedpass_reading,
                                  'Passing Math': groupedpass_math})

#Merge dataframes based on common school name
finschool_summary_df = pd.merge(school_summary_df, reduced_school_df, on = "school_name")

#Calculate passing math and reading %s and add to dataframe 
percent_math = finschool_summary_df['Passing Math']/finschool_summary_df['size']*100
percent_reading = finschool_summary_df['Passing Reading']/finschool_summary_df['size']*100
finschool_summary_df["% Passing Math"] = percent_math
finschool_summary_df["% Passing Reading"] = percent_reading

#Calculate overall passing % (passed both math and reading)
#Group dataframe from District Summary by school name and count students who passed both subjects
groupedpassed_merge_df = passed_merge_df.groupby('school_name_x')
passedboth = groupedpassed_merge_df['school_name_x'].count()
#Create new dataframe of counted values and merge with main dataframe 
finpassed_both_df = pd.DataFrame(passedboth)
renamedpassed_both_df = finpassed_both_df.rename(columns = {'school_name_x':'Passed Both'})
mergeschool_summary_df = pd.merge(finschool_summary_df , renamedpassed_both_df , left_on = 'school_name', right_on = 'school_name_x')
#Calculate % of overall passing 
percent_both = mergeschool_summary_df['Passed Both']/mergeschool_summary_df['size']*100
mergeschool_summary_df["% Overall Passing"] = percent_both

#Select necessary columns 
new_columns = ['school_name','type','size','budget','Per Student Budget','Average Math Score', 'Average Reading Score',
          '% Passing Math', '% Passing Reading', '% Overall Passing']
newschool_summary_df  = mergeschool_summary_df[new_columns]

#Rename columns 
new_names = {"type": "School Type", "size": "Total Students","budget": "Total School Budget"}
renamed_schoolsummary_df = newschool_summary_df.rename ( columns = new_names)

#Apply displayed formatting via mapping 
renamed_schoolsummary_df['Per Student Budget'] = renamed_schoolsummary_df['Per Student Budget'].astype(int) 
renamed_schoolsummary_df['Total School Budget'] = renamed_schoolsummary_df['Total School Budget'].astype(int)
renamed_schoolsummary_df["Total School Budget"] = renamed_schoolsummary_df["Total School Budget"].map("${:,.2f}".format)
renamed_schoolsummary_df["Per Student Budget"] = renamed_schoolsummary_df["Per Student Budget"].map("${:,.2f}".format)

#Set school name as index 
renamed_schoolsummary_df.set_index('school_name', inplace=True)
renamed_schoolsummary_df.index.names = ['']
renamed_schoolsummary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [7]:
#Top Performing Schools (By % Overall Passing)
#Sort and display top five performing schools by % overall passing
top_schools_df = renamed_schoolsummary_df.sort_values('% Overall Passing',ascending=False).head(5)
top_schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [8]:
#Bottom Performing Schools (By % Overall Passing)
#Sort and display five worst-performing schools by % overall passing
bottom_schools_df = renamed_schoolsummary_df.sort_values('% Overall Passing',ascending=True).head(5)
bottom_schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [9]:
#Math Scores by Grade
#Create copy
grades_df = student_df.copy()

#Per grade level, group by school name and find average 
ninth_grade = grades_df.loc[grades_df['grade'] == '9th'].groupby('school_name').mean()
tenth_grade_df = grades_df.loc[grades_df['grade'] == '10th'].groupby('school_name').mean()
eleventh_grade_df = grades_df.loc[grades_df['grade'] == '11th'].groupby('school_name').mean()
twelfth_grade_df = grades_df.loc[grades_df['grade'] == '12th'].groupby('school_name').mean()

#Create dataframe 
grade_summary_df = pd.DataFrame(ninth_grade)
grade_summary_df
new_names9 = {"math_score": "9th_math", "reading_score": "9th_reading"}
renamedgrade_summary_df = grade_summary_df.rename ( columns = new_names9)

mergedgrade_summary_df = renamedgrade_summary_df.merge(tenth_grade_df , on="school_name") 
mergedgrade_summary_df
new_names10 = {"math_score": "10th_math", "reading_score": "10th_reading"}
mergedgrade_summary_df = mergedgrade_summary_df.rename ( columns = new_names10)

mergedgrade_summary_df = mergedgrade_summary_df.merge(eleventh_grade_df , on="school_name") 
new_names11 = {"math_score": "11th_math", "reading_score": "11th_reading"}
mergedgrade_summary_df = mergedgrade_summary_df.rename ( columns = new_names11)

mergedgrade_summary_df = mergedgrade_summary_df.merge(twelfth_grade_df , on="school_name") 
new_names12 = {"math_score": "12th_math", "reading_score": "12th_reading"}
mergedgrade_summary_df = mergedgrade_summary_df.rename ( columns = new_names12)

#delete student ID columns 
del mergedgrade_summary_df['Student ID_x']
del mergedgrade_summary_df['Student ID_y']

#Pull only math columns 
new_columns = ['9th_math','10th_math','11th_math','12th_math']
math_scores_df  = mergedgrade_summary_df[new_columns]
new_names = {"9th_math": "9th", "10th_math": "10th","11th_math": "11th","12th_math": "12th"}
math_scores_df  = math_scores_df.rename(columns = new_names)
math_scores_df.index.names = ['']
math_scores_df

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [10]:
#Reading Scores by Grade
#Pull only reading columns from dataframe
new_columns = ['9th_reading','10th_reading','11th_reading','12th_reading']
reading_scores_df  = mergedgrade_summary_df[new_columns]
new_names = {"9th_reading": "9th", "10th_reading": "10th","11th_reading": "11th","12th_reading": "12th"}
reading_scores_df  = reading_scores_df.rename(columns = new_names)
reading_scores_df.index.names = ['']
reading_scores_df

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [11]:
#Scores by School Spending
#Create bins and their names respectively 
bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

#Create copy and place into bins  
budget_df = newschool_summary_df.copy()
budget_df["Spending Ranges (Per Student)"] = pd.cut(budget_df["Per Student Budget"], bins, labels = group_names)

#Pull specific columns to a reduced dataframe 
n_columns = ['Spending Ranges (Per Student)','Average Math Score','Average Reading Score',
                '% Passing Math','% Passing Reading','% Overall Passing']
reduced_budget_df = budget_df[n_columns]

#Create groups based off bins and calculate average scores 
grouped_budget_df = reduced_budget_df.groupby('Spending Ranges (Per Student)').mean()

#Apply displayed formatting via mapping 
grouped_budget_df['Average Math Score'] = grouped_budget_df['Average Math Score'].map("{:,.2f}".format)
grouped_budget_df['Average Reading Score'] = grouped_budget_df['Average Reading Score'].map("{:,.2f}".format)
grouped_budget_df['% Passing Math'] = grouped_budget_df['% Passing Math'].map("{:,.2f}".format)
grouped_budget_df['% Passing Reading'] = grouped_budget_df['% Passing Reading'].map("{:,.2f}".format)
grouped_budget_df['% Overall Passing'] = grouped_budget_df['% Overall Passing'].map("{:,.2f}".format)
grouped_budget_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


In [12]:
#Scores by School Size
#Create bins and their names respectively 
bins = [0, 999, 1999, 5000]
group_names = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

#Create copy and place into bins 
size_df = newschool_summary_df.copy()
size_df["School Size"] = pd.cut(size_df["size"], bins, labels = group_names)

#Create groups based off bins and calculate average scores  
grouped_size_df = size_df.groupby('School Size').mean()
grouped_size_df = grouped_size_df[["Average Math Score", "Average Reading Score", "% Passing Math", 
                             "% Passing Reading", "% Overall Passing"]]
grouped_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [13]:
#Scores by School Type
#Create copy 
type_df = newschool_summary_df.copy()

#Pull specific columns to a reduced dataframe and rename 
t_columns = ['type','Average Math Score','Average Reading Score',
                 '% Passing Math','% Passing Reading','% Overall Passing']
prereduced_type_df = type_df[t_columns]
reduced_type_df = prereduced_type_df.rename(columns = {"type": "School Type"})

#Create groups and calculate average scores 
grouped_type_df = reduced_type_df.groupby('School Type').mean()
grouped_type_df = grouped_type_df[["Average Math Score", "Average Reading Score","% Passing Math","% Passing Reading", "% Overall Passing"]]
grouped_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [ ]:
#Observable trends based on data

# Based on the PyCitySchool analysis using Pandas, charter schools have drastically higher standardized math  
    # and reading test compared to the district schools. 
# When using school size as a comparison factor, larger schools with student population of greater than 2000 
    # tend to have lower percentage of students who pass both standardized math and science tests compared to 
    # small and medium student population schools. 

# It is interesting to note that an increase in budget spending per student does not correlate to higher math 
    # and reading test pass rates. There is an inverse/negative correlation between spending per student and 
    # test pass rates. If the priority of these schools were to increase test pass rates, the data trend aforementioned 
    # may assist the school board and the major to consider reallocating and reprioritizing future school budgets 
    # that directly increase the chances of students’ success in math and reading subjects. 

# There is little difference in the average of math and test scores in each grade year per school. As students 
    # progress from 9th to 12th grade, it could be theorized that the average test scores may decrease as the 
    # difficulty of the subject matter increases. However, this is not seen in the data output in Math and Reading 
    # Scores by Grade. 
